In [ ]:
from Environment.custom_env import CustomEnvironment
import json
import torch
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook

from Models.ActorCritic import ActorCritic
from Models.SoftActorCritic import SoftActorCritic
from Models.DDQN import DoubleDQN

In [ ]:
torch.cuda.is_available()

In [ ]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

In [ ]:
env = CustomEnvironment(env_config=env_variables, render_mode="human")
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 128))

In [ ]:
ac_config = open('Environment/ac_config.json')
ac_variables = json.load(ac_config)
ac_config.close

sac_config = open('Environment/sac_config.json')
sac_variables = json.load(sac_config)
sac_config.close

ddqn_config = open('Environment/ddqn_config.json')
ddqn_variables = json.load(ddqn_config)
ddqn_config.close

In [ ]:
ddqn = DoubleDQN(ddqn_variables)
# ac = ActorCritic(ac_variables)
# sac = SoftActorCritic(sac_variables)

In [ ]:
models = {0:ddqn, 1:ddqn, 2:ddqn}

In [ ]:
episodes = 1000

for episode in tqdm_notebook(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",

    losses = {}
    while(env.agents): #until there are any surviving agents 

        actions = {}
        log_probs = {}

        for a in env.agents:
            # print(curr_obs[a])
            act, log_prob = models[a].select_action(state = torch.tensor(curr_obs[a], dtype = torch.float32))
            actions[a] = act
            log_probs[a] = log_prob

        # for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
        #     if i in env.agents:
        #         act, log_prob = env.agents_objects[i].brain1.select_action(state = torch.tensor(curr_obs[i], dtype = torch.float32))
        #         actions[i] = act
        #         log_probs[i] = log_prob
        #     else:
        #         actions[i] = env.num_actions
        #         log_probs[i] = 69
            
        # print(actions)

        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        if env.timestep % 50 == 0:
            print(rewards)
        
        # print(next_obs)
        # print(env.agents_objects)
        # print(curr_obs)
        # print(rewards)
        # print(terminations)
        # print(truncations)
        # print(infos)

        for id, obs in next_obs.items():
            models[id].push_to_buffer(torch.tensor(curr_obs[id], dtype = torch.float32),actions[id],rewards[id],torch.tensor(obs, dtype = torch.float32),log_probs[id],terminations[id])
            loss = models[id].update_weights()

            if(terminations[id] == True):
                # loss = models[id].update_weights()
                losses[id] = loss
        
        # for i in range(env.n_agents):

        #     if i in env.agents:
        #         env.agents_objects[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         env.agents_objects[i].brain1.update_weights() 
            
        #     if i in env.justdie:
        #         env.justdie[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         loss = env.justdie[i].brain1.update_weights() 
        #         del env.justdie[i]
        #         losses[i] = loss
        
        curr_obs = next_obs

    print(f"Episode {episode} end")
    print(f"Losses {losses}")
    print()
    env.close()

In [ ]:
models[0].steps_done